# Modeling

### ResNet 

In [1]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import sys
import gc
import random
import time
from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter

import skimage.io
import cv2 
from PIL import Image
import scipy as sp

import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from functools import partial
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
import torchvision.models as models
import torch.cuda

from albumentations import Compose, Transpose, Normalize, HorizontalFlip, VerticalFlip
from albumentations.pytorch import ToTensorV2

from collections import OrderedDict
import math
from sklearn.metrics import cohen_kappa_score
from warmup_scheduler import GradualWarmupScheduler
import warnings 
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda')

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
DEBUG = True
epochs = 1 if DEBUG else 20
df_train = df_train.sample(100).reset_index(drop=True) if DEBUG else df_train
height=256
width=256
lr=1e-4
batch_size=2
fold=0
image_size=256
tile_size=256
#epochs=20
seed=42
n_tiles=12
num_workers=4
warmup_factor=10
target_size=6 #1
target_col='isup_grade'
n_fold=4 

In [4]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('PANDA')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'train.log'
LOGGER = init_logger(LOG_FILE)


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)


In [5]:
def get_tiles(img):
        tiles = []
        H, W, C = img.shape
        #1
        pad_H = (tile_size - H % tile_size) % tile_size 
        pad_W = (tile_size - W % tile_size) % tile_size 
        #2
        img2 = np.pad(img,[[pad_H // 2, pad_H - pad_H // 2], [pad_W // 2,pad_W - pad_W//2], 
                            [0,0]], constant_values=255)
        
        #3
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3)
        #4
        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        #5
        if len(img3) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        #6
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            tiles.append({'img':img3[i], 'idx':i})
        return tiles, n_tiles_with_info >= n_tiles 

In [6]:
class Dataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join('train_images', f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                if self.transform is not None:
                    this_img = self.transform(image=this_img)['image']
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

        if self.transform is not None:
            images = self.transform(image=images)['image']
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        label = np.zeros(5).astype(np.float32)
        label[:row.isup_grade] = 1.
        return torch.tensor(images), torch.tensor(label)

In [7]:
def get_transforms(*, data):
    
    assert data in ('train', 'valid')
    
    if data == 'train':
        return Compose([
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),])
    
    elif data == 'valid':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),])

In [8]:
skf = StratifiedKFold(4, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

,image_id,data_provider,isup_grade,gleason_score,fold
0,777e89beb78fc98db5806423fe7f7254,radboud,5,5+4,0
1,b1fbe9701b14bf285318abcbf002d5ea,radboud,4,4+4,3
2,441b527337da4319cf700b6e7da9fa0c,radboud,1,3+3,2
3,9d931784c507a8633fdc623cc916d8d3,karolinska,0,0+0,0
4,d13eecf391803d164d6f0fd89f1871e8,radboud,1,3+3,0


In [13]:
def data_setup():
    
    train_idx = np.where((df_train['fold'] != fold))[0]
    valid_idx = np.where((df_train['fold'] == fold))[0]

    global dataset_train, dataset_valid, train_loader, valid_loader, df_valid, df_this

    df_this  = df_train.loc[train_idx]
    df_valid = df_train.loc[valid_idx]

    dataset_train = Dataset(df_this , image_size, n_tiles, transform=get_transforms(data='train'))
    dataset_valid = Dataset(df_valid, image_size, n_tiles, transform=get_transforms(data='valid'))
    
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)

    return 

def build_the_network():
    
    global model
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
                      nn.Linear(num_ftrs, 5))
    model = model.to(device) 
    
    return 

def optim():
    
    global optimizer, scheduler, criterion
    optimizer = Adam(model.parameters(), lr=lr/warmup_factor, amsgrad=False)
#scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, eps=1e-6)
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs-1)
    scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, 
                                   total_epoch=1, 
                                   after_scheduler=scheduler_cosine)
    criterion = nn.BCEWithLogitsLoss() 
    
    return 

def train_epoch(loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        
        data, target = data.to(device), target.to(device)
        loss_func = criterion
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        loss.backward()
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss


def val_epoch(loader, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)
            loss = criterion(logits, target)

            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target.sum(1))

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'karolinska'], df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
    qwk_r = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'radboud'], df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
    print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return PREDS
    else:
        return PREDS,TARGETS,LOGITS, val_loss, acc, qwk

In [14]:
data_setup()
build_the_network()
optim()

qwk_max = 0.
best_file = 'best_resnet50_tiled.pth'
for epoch in range(1, epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)

    train_loss = train_epoch(train_loader, optimizer)
    PREDS, TARGETS, LOGITS, val_loss, acc, qwk = val_epoch(valid_loader)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}, qwk: {(qwk):.5f}'
    print(content)

    if qwk > qwk_max:
        print('score progress: ({:.6f} --> {:.6f}).  Saving model ...'.format(qwk_max, qwk))
        torch.save(model.state_dict(), best_file)
        qwk_max = qwk 

torch.save(model.state_dict(), os.path.join('final_resnet50_tiled.pth'))

  0%|          | 0/38 [00:00<?, ?it/s]

Sat Sep 12 15:47:32 2020 Epoch: 1


100%|██████████| 13/13 [00:03<00:00,  3.49it/s]


qwk 0.0 qwk_k 0.0 qwk_r 0.0
Sat Sep 12 15:47:55 2020 Epoch 1, lr: 0.0000100, train loss: 0.66233, val loss: 0.62946, acc: 4.00000, qwk: 0.00000


In [15]:
print(PREDS)
print(TARGETS)
print(LOGITS)
cohen_kappa_score(PREDS, TARGETS)
#type(PREDS)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2.]
[5. 0. 1. 5. 1. 2. 0. 1. 3. 4. 1. 1. 0. 1. 0. 1. 0. 5. 1. 5. 3. 0. 4. 5.
 0.]
[[ 0.25989306 -1.063205   -0.99710244 -1.4471527  -0.7404163 ]
 [ 0.2596209  -1.0972604  -0.98380655 -1.4158733  -0.75114566]
 [ 0.25071204 -1.0730724  -0.9860933  -1.4308074  -0.7395149 ]
 [ 0.26247847 -1.0713757  -1.0024192  -1.4485155  -0.74612516]
 [ 0.26025403 -1.0894552  -0.9963755  -1.4262763  -0.76121145]
 [ 0.2514897  -1.0979496  -1.0060452  -1.4589238  -0.76208246]
 [ 0.2589507  -1.1040052  -1.0133222  -1.4609251  -0.7720044 ]
 [ 0.24373199 -1.1047088  -0.9778441  -1.4185582  -0.746756  ]
 [ 0.265221   -1.1161801  -1.0056255  -1.4410576  -0.7686835 ]
 [ 0.25395536 -1.0690111  -1.0166548  -1.4520988  -0.7558238 ]
 [ 0.25793177 -1.1046823  -0.99891144 -1.4469073  -0.76737475]
 [ 0.2600941  -1.0853918  -1.013014   -1.4671478  -0.7727766 ]
 [ 0.25302732 -1.0917164  -1.0035732  -1.4555908  -0.7576212 ]
 [ 0.2605924  -1.1017995 

0.0

In [ ]:
cohen_kappa_score([2.,2.,2.],[0.,4.,0.])

In [ ]:
sys.path = ['best_resnet50_tiled.pth'] + sys.path
model_dict = {'resnet50': 'best_resnet50_tiled.pth'}
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_ftrs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.5),
                      nn.Linear(256, 5),                   
                      nn.Softmax()
model.load_state_dict(torch.load(model_dict['resnet50']))
#model.to(device)
model.eval() 

In [ ]:
print(model)
model_weights = [] 
conv_layers = [] 
model_children = list(model.children())
model_weights

In [ ]:
# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolutional layers: {counter}")

In [ ]:
# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")

In [ ]:
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
    plt.imshow(filter[0, :, :].detach(), cmap='gray')
    plt.axis('off')
    plt.savefig('filter0.png')
plt.show() 

In [ ]:
# read and visualize an image
image = skimage.io.MultiImage(os.path.join("train_images",
                                           '08f055372c7b8a7e1df97c6586542ac8.tiff'))

image = cv2.resize(image[-1], (height, width))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()
# define the transforms
transform = Compose([HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
       Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],),
           ToTensorV2()])
image = np.array(image)
# apply the transforms
augmented = transform(image=image)
print(type(augmented))
image = augmented['image']
print(image.size())
# unsqueeze to add a batch dimension
image = image.unsqueeze(0)
print(image.size()) 

In [ ]:
# pass the image through all the layers
results = [conv_layers[0](image)]
for i in range(1, len(conv_layers)):
    # pass the result from the last layer to the next layer
    results.append(conv_layers[i](results[-1]))
# make a copy of the `results`
outputs = results

In [ ]:
# visualize 64 features from each layer 
# (although there are more feature maps in the upper layers)
for num_layer in range(len(outputs)):
    plt.figure(figsize=(30, 30))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print(layer_viz.size())
    for i, filter in enumerate(layer_viz):
        if i == 64: # we will visualize only 8x8 blocks from each layer
            break
        plt.subplot(8, 8, i + 1)
        plt.imshow(filter, cmap='gray')
        plt.axis('off')
    #print(f'Saving layer {num_layer} feature maps...')
    plt.savefig(f'resnet50_tiled_feature_maps/outputs_layer_{num_layer}.png') 
    plt.show()
    plt.close()